## Vocabulario del problema

### 1.Cargar archivos

In [6]:
import pandas as pd
import os
ruta_archivo = os.path.join("imdb_dataset.csv")
df_criticas = pd.read_csv(ruta_archivo, encoding="ISO-8859-1").sample(10000, replace=False)
df_criticas

,Review,Label
24800,"A film for mature, educated audiences...<br />...",pos
9274,"This has to be one of, if not THE greatest Mob...",neg
17172,The Lone Ranger was one of my childhood heroes...,pos
23313,Jamie Foxx did an incredible job playing Ray C...,pos
17896,Captain Corelli's Mandolin is one of Nicolas C...,pos
...,...,...
47539,This movie captures the essence of growing up ...,pos
42944,"Sometimes, Lady Luck smiles on me. I had origi...",pos
48238,"If ""The Cabinet of Dr. Caligari"" is the father...",pos
29938,Two qualifiers right up front: I actually thin...,neg


### 2. Palabras de parada

In [9]:
import nltk
import string
palabras_de_parada_ingles = set( nltk.corpus.stopwords.words('english') + list(string.punctuation))
palabras_de_parada_ingles

### 3. Normalización

In [10]:
def normalizar_critica(critica, palabras_de_parada_ingles, language='english'):
    tokens = nltk.word_tokenize(critica,language)
    tokens_filtrados = [token.lower() for token in tokens if token not in palabras_de_parada_ingles]
    return " ".join(tokens_filtrados)

In [ ]:
def normalizar_cuerpo_texto(criticas, palabras_de_parada_ingles):
    
    
    
    #return np.array([normalizar_oraciones(oracion, language) for oracion in oraciones])

df_criticas_normalizado = normalizar_cuerpo_texto(df_criticas, palabras_de_parada_ingles)
df_criticas_normalizado